# Goal

The goal is to build a visualization of the graph generated in the previous files. 
The choice for the visualization is to use Neo4j Aura. 

# How to run this file

Required: 
- A Neo4j Aura instance

Your secrets (API keys) must be put in a ".env" file in the same folder as this file.

In [1]:
import os
from dotenv import load_dotenv
from langchain.graphs import Neo4jGraph
load_dotenv()

graph = Neo4jGraph(
    url=os.getenv('NEO4J_URL'),
    username=os.getenv('NEO4J_USERNAME'),
    password=os.getenv('NEO4J_PASSWORD')
)


# Retrieve all nodes and relationships from the SQLite DB

In [2]:
# Retrieve the actors and relationships that have been stored in the local SQLite DB

import sqlite3

def retrieve_actors_from_sqlite(db_name="streetpress.db"):
    # Connect to the db
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    query = "SELECT * FROM actors"
    cursor.execute(query)

    # Fetch and return the result
    result = cursor.fetchall()

    # Close the connection
    conn.close()

    return result


def retrieve_relationships_from_sqlite(db_name="streetpress.db"):
    # Connect to the db
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    query = "SELECT * FROM relationships"
    cursor.execute(query)

    # Fetch and return the result
    result = cursor.fetchall()

    # Close the connection
    conn.close()

    return result

# Create a GraphDocument from nodes and relationships

We are using Langchain to manage the interface with Neo4j. 
Langchain requires a GraphDocument, made of Node and Relationship objects.
We build the GraphDocument from the actors and relationships retrieved from SQLite.

In [3]:
from langchain.graphs.graph_document import GraphDocument, Node, Relationship
from langchain_core.documents.base import Document

# Create the GraphDocument object
def create_graph_document(nodes, rels):
    graph_nodes = []
    graph_rels = []
    id_map = {}

    for node in nodes:
        graph_node = Node(
            id= node[0],
            type= "Actor",
            properties= {
                "name": node[1],
                "label": node[2],
                "key_information": node[3]
            }
        )
        graph_nodes.append(graph_node)
        id_map[node[0]]=graph_node

    for rel in rels:
        graph_rel = Relationship(
            source= id_map[rel[1]],
            target= id_map[rel[2]],
            type= rel[3],
            properties= {
                "rationale": rel[4]
            }
        )
        graph_rels.append(graph_rel)

    graph_document = GraphDocument(
        nodes= graph_nodes,
        relationships= graph_rels,
        source= Document(page_content="Streetpress")
    )

    return graph_document

# Delete the content of the Neo4j Aura database
def delete_graph_from_aura(neo4jgraph):
    neo4jgraph.query("MATCH (n) DETACH DELETE n")


# Put it all together

In [4]:
# Retrieve the data from the SQLite DB
nodes = retrieve_actors_from_sqlite()
rels = retrieve_relationships_from_sqlite()

# Build a graph document
graph_document = create_graph_document(nodes, rels)

# Delete the exiting content of Neo4j Aura before adding new content
delete_graph_from_aura(graph)

# Insert all the nodes and relationships from the graph document into Neo4j Aura
graph.add_graph_documents([graph_document])

In [5]:
delete_graph_from_aura(graph)